In [1]:
# Importing stock information from Yahoo finance 

import yfinance as yf
import pandas as pd





In [2]:
#Making empty for storing financial information

df_financial = pd.read_csv('Tickers.csv', sep=';')
df_financial.head()

,Company,Ticker
0,ABNAMRO,ABN.AS
1,Agrifirm,0
2,AgroFair,0
3,Amazon,AMZN
4,Armor,0


In [3]:
#Removing companies without tickers

df_financial = df_financial[(df_financial['Ticker'])!='0'].reset_index(drop=True)
df_financial.head()

,Company,Ticker
0,ABNAMRO,ABN.AS
1,Amazon,AMZN
2,asr,ASRNL.AS
3,bam,BAMNB.AS
4,CocaCola,KO


In [4]:
yf.download('KO','2020-05-11',progress=False)['Adj Close'].values[0]

43.0617561340332

In [5]:
# (Denne tar ca. 12 min. Så skal få lagret innhold til CSV når vi endelig har bestemt selskap)
# Adding sector, exchange and currency information from Yahoo to df for each stock

df_financial['Sector'] = df_financial['Ticker'].apply(lambda x: yf.Ticker(x).info['sector'])
df_financial['Exchange'] = df_financial['Ticker'].apply(lambda x: yf.Ticker(x).info['exchange'])
df_financial['Currency'] = df_financial['Ticker'].apply(lambda x: yf.Ticker(x).info['currency'])

df_financial.head()

,Company,Ticker,Sector,Exchange,Currency
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR
1,Amazon,AMZN,Consumer Cyclical,NMS,USD
2,asr,ASRNL.AS,Financial Services,AMS,EUR
3,bam,BAMNB.AS,Industrials,AMS,EUR
4,CocaCola,KO,Consumer Defensive,NYQ,USD


In [6]:
# Adding adjusted close values (Takes into account dividend and stock splits)

df_financial['adjusted_10.05.2020'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2020-05-11',progress=False)['Adj Close'].values[0])
df_financial['adjusted_10.05.2022'] = df_financial['Ticker'].apply(lambda x: yf.download(x, '2022-05-11',progress=False)['Adj Close'].values[0])

In [7]:
#Using dictionary to map stock exchange to ticker

Ticker_exchange = {'NYQ': '^NYA', 'AMS': '^AEX', 'NMS': '^IXIC', 'LSE':'^FTSE' ,
 'STO':'^OMX', 'PNK':'^AEX','EBS':'^SSMI', 'GER':'^GDAXI', 'CPH':'^OMXC20'}


df_financial['Exchange_ticker'] = df_financial['Exchange'].map(Ticker_exchange)
df_financial.head()

,Company,Ticker,Sector,Exchange,Currency,adjusted_10.05.2020,adjusted_10.05.2022,Exchange_ticker
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR,6.283122,11.105000,^AEX
1,Amazon,AMZN,Consumer Cyclical,NMS,USD,2409.000000,2177.179932,^IXIC
2,asr,ASRNL.AS,Financial Services,AMS,EUR,19.876909,42.849998,^AEX
3,bam,BAMNB.AS,Industrials,AMS,EUR,1.478000,2.402000,^AEX
4,CocaCola,KO,Consumer Defensive,NYQ,USD,43.061756,64.010002,^NYA


In [8]:
# Adding exchange prices to df

df_financial['Exchange_2020.05.10'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2020-05-10',progress=False).Close.values[0])
df_financial['Exchange_2022.05.10'] = df_financial['Exchange_ticker'].apply(lambda x: yf.download(x, '2022-05-10',progress=False).Close.values[0])

df_financial.head()

,Company,Ticker,Sector,Exchange,Currency,adjusted_10.05.2020,adjusted_10.05.2022,Exchange_ticker,Exchange_2020.05.10,Exchange_2022.05.10
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR,6.283122,11.105000,^AEX,520.330017,669.739990
1,Amazon,AMZN,Consumer Cyclical,NMS,USD,2409.000000,2177.179932,^IXIC,9192.339844,11623.250000
2,asr,ASRNL.AS,Financial Services,AMS,EUR,19.876909,42.849998,^AEX,520.330017,669.739990
3,bam,BAMNB.AS,Industrials,AMS,EUR,1.478000,2.402000,^AEX,520.330017,669.739990
4,CocaCola,KO,Consumer Defensive,NYQ,USD,43.061756,64.010002,^NYA,11281.370117,15057.870117


In [9]:
#Using dictionary to map currency to the Yahoo searchword for exchange rate

Valuta = {'EUR': 'EURUSD=x', 'DKK': 'DKKUSD=x', 'SEK': 'SEKUSD=x', 'CHF':'CHFUSD=x' ,
 'GBp':'GBPUSD=x', 'USD': 'USD=x'}


df_financial['valuta_ticker'] = df_financial['Currency'].map(Valuta)

In [10]:
# Correcting for exchange rates for stock and stock exchange prices

df_financial['Exchange_rate_2020'] = df_financial['valuta_ticker'].apply(lambda x: yf.download(x, '2020-05-10', progress=False).Close[0])
df_financial['Exchange_rate_2022'] = df_financial['valuta_ticker'].apply(lambda x: yf.download(x, '2022-05-10', progress=False).Close[0])
df_financial['USD_adjusted_2020.10.05'] = df_financial['Exchange_rate_2020']*df_financial['adjusted_10.05.2020']
df_financial['USD_adjusted_2022.10.05'] = df_financial['Exchange_rate_2022']*df_financial['adjusted_10.05.2022']
df_financial['USD_Exchange_2020.10.05'] = df_financial['Exchange_rate_2020']*df_financial['Exchange_2020.05.10']
df_financial['USD_Exchange_2022.10.05'] = df_financial['Exchange_rate_2022']*df_financial['Exchange_2022.05.10']
df_financial.head()


,Company,Ticker,Sector,Exchange,Currency,adjusted_10.05.2020,adjusted_10.05.2022,Exchange_ticker,Exchange_2020.05.10,Exchange_2022.05.10,valuta_ticker,Exchange_rate_2020,Exchange_rate_2022,USD_adjusted_2020.10.05,USD_adjusted_2022.10.05,USD_Exchange_2020.10.05,USD_Exchange_2022.10.05
0,ABNAMRO,ABN.AS,Financial Services,AMS,EUR,6.283122,11.105000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.053219,6.810603,11.695999,564.012799,705.383016
1,Amazon,AMZN,Consumer Cyclical,NMS,USD,2409.000000,2177.179932,^IXIC,9192.339844,11623.250000,USD=x,1.000000,1.000000,2409.000000,2177.179932,9192.339844,11623.250000
2,asr,ASRNL.AS,Financial Services,AMS,EUR,19.876909,42.849998,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.053219,21.545617,45.130441,564.012799,705.383016
3,bam,BAMNB.AS,Industrials,AMS,EUR,1.478000,2.402000,^AEX,520.330017,669.739990,EURUSD=x,1.083952,1.053219,1.602081,2.529832,564.012799,705.383016
4,CocaCola,KO,Consumer Defensive,NYQ,USD,43.061756,64.010002,^NYA,11281.370117,15057.870117,USD=x,1.000000,1.000000,43.061756,64.010002,11281.370117,15057.870117
